In [1]:
BUCKET_NAME='test-xwh01'
OBS_BASE_PATH=BUCKET_NAME

from modelarts.session import Session
session = Session()

session.download_data(bucket_path="test-xwh01/test-machine/ratings.dat", path="./ratings.dat")
session.download_data(bucket_path="test-xwh01/test-machine/users.dat", path="./users.dat")
session.download_data(bucket_path="test-xwh01/test-machine/movies.dat", path="./movies.dat")
# import same usefull libraries
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
movie_rating = pd.read_csv('./ratings.dat')

Successfully download file test-xwh01/test-machine/ratings.dat from OBS to local ./ratings.dat
Successfully download file test-xwh01/test-machine/users.dat from OBS to local ./users.dat
Successfully download file test-xwh01/test-machine/movies.dat from OBS to local ./movies.dat


In [2]:
movie_rating_cols = ['user_id',  'movie_id', 'rating', 'unix_tiemstamp'] 
movie_rating = pd.read_csv('./ratings.dat',sep = '::', names = movie_rating_cols, parse_dates = True)
movie_rating.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,user_id,movie_id,rating,unix_tiemstamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
movie_rating.nunique()

user_id             6040
movie_id            3706
rating                 5
unix_tiemstamp    458455
dtype: int64

In [4]:
user=pd.read_csv('./users.dat',sep='::',names=['UserID','Gender','Age','Occupation','Zip-code'])
user.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
user.shape

(6040, 5)

In [6]:
movie=pd.read_csv('./movies.dat',sep='::',names=['MovieID','Title','Genres'])
movie.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movie.shape

(3883, 3)

In [8]:
movie_rating.drop("unix_tiemstamp",inplace = True, axis = 1)
movie_rating.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [9]:
# 拆分训练集与数据集
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(movie_rating,test_size = 0.2)

In [10]:
# 创建用户电影矩阵
movie_rating_matrix = train_data.pivot_table(index = ['movie_id'], columns = ['user_id'],
                                              values = 'rating').reset_index(drop = True)
movie_rating_matrix.fillna(0, inplace = True)
cmu = movie_rating_matrix
cmu.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# 利用余弦算法获得电影相似度矩阵
movie_similarity = 1 - pairwise_distances(movie_rating_matrix.values, metric = "cosine")
np.fill_diagonal(movie_similarity,0)
# 将相似度填回相似矩阵
movie_rating_matrix = pd.DataFrame(movie_similarity)
movie_rating_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,3668,3669,3670,3671,3672,3673,3674,3675,3676,3677
0,0.000000,0.301448,0.231415,0.143140,0.202166,0.272934,0.239149,0.095466,0.090785,0.305364,...,0.073091,0.024195,0.069109,0.071217,0.040325,0.255665,0.138251,0.074884,0.043667,0.141330
1,0.301448,0.000000,0.193094,0.126886,0.216710,0.200680,0.205366,0.164041,0.130004,0.308755,...,0.038133,0.023976,0.051388,0.083308,0.038894,0.160515,0.117904,0.096252,0.014369,0.091311
2,0.231415,0.193094,0.000000,0.132988,0.216779,0.154236,0.218045,0.085001,0.102843,0.186565,...,0.034003,0.051744,0.034102,0.073484,0.040583,0.149687,0.062150,0.034142,0.007601,0.064717
3,0.143140,0.126886,0.132988,0.000000,0.216605,0.097738,0.162099,0.038360,0.042244,0.109251,...,0.031211,0.000000,0.002676,0.011158,0.009768,0.078763,0.050529,0.012975,0.008233,0.065474
4,0.202166,0.216710,0.216779,0.216605,0.000000,0.124719,0.245664,0.111330,0.098363,0.193229,...,0.026511,0.054450,0.051153,0.043877,0.007779,0.141991,0.083355,0.045158,0.008741,0.080878


In [12]:
# 获得TOP-N推荐
user_inp = "Toy Story (1995)"
inp = movie[movie['Title'] == user_inp].index.tolist()
inp = inp[0]
movie['similarity'] = movie_rating_matrix.iloc[inp]
movie.columns = ['MovieID','Title','Genres','similarity']
movie.head()

,MovieID,Title,Genres,similarity
0,1,Toy Story (1995),Animation|Children's|Comedy,0.000000
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.301448
2,3,Grumpier Old Men (1995),Comedy|Romance,0.231415
3,4,Waiting to Exhale (1995),Comedy|Drama,0.143140
4,5,Father of the Bride Part II (1995),Comedy,0.202166


In [13]:
# 测试集合构造
rating_test_matrix = test_data.pivot_table(index = ['movie_id'],columns = ['user_id'],
                                values = 'rating').reset_index(drop = True)
rating_test_matrix.fillna(0,inplace = True)
cmu_test = rating_test_matrix
cmu_test.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# 测试集构造物品相似矩阵
movie_similarity_test = 1 - pairwise_distances(rating_test_matrix.values, metric = "cosine")
np.fill_diagonal(movie_similarity_test,0)
rating_matrix_test = pd.DataFrame(movie_similarity_test)
rating_matrix_test.head()

,0,1,2,3,4,5,6,7,8,9,...,3457,3458,3459,3460,3461,3462,3463,3464,3465,3466
0,0.000000,0.075271,0.067764,0.038394,0.060639,0.099744,0.041919,0.060896,0.022563,0.087924,...,0.000000,0.008774,0.00972,0.020920,0.015644,0.094506,0.036423,0.0,0.009600,0.032957
1,0.075271,0.000000,0.061038,0.013833,0.051253,0.043864,0.036553,0.079374,0.081293,0.085681,...,0.009134,0.000000,0.00000,0.008479,0.000000,0.042320,0.018627,0.0,0.000000,0.000000
2,0.067764,0.061038,0.000000,0.042493,0.048304,0.017406,0.041148,0.051088,0.053631,0.065591,...,0.007652,0.000000,0.00000,0.000000,0.047220,0.040343,0.043621,0.0,0.017386,0.011300
3,0.038394,0.013833,0.042493,0.000000,0.009981,0.065456,0.034700,0.000000,0.030869,0.031164,...,0.000000,0.000000,0.00000,0.014311,0.000000,0.049257,0.017147,0.0,0.000000,0.042026
4,0.060639,0.051253,0.048304,0.009981,0.000000,0.015740,0.037086,0.000000,0.013197,0.053557,...,0.000000,0.153960,0.00000,0.000000,0.000000,0.021320,0.005498,0.0,0.000000,0.055021


In [15]:
# 测试集TOP-N推荐
user_inp = "Toy Story (1995)"
inp = movie[movie['Title'] == user_inp].index.tolist()
inp = inp[0]
movie['similarity'] = rating_matrix_test.iloc[inp]
movie.columns = ['MovieID','Title','Genres','similarity']
movie.head()

,MovieID,Title,Genres,similarity
0,1,Toy Story (1995),Animation|Children's|Comedy,0.000000
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.075271
2,3,Grumpier Old Men (1995),Comedy|Romance,0.067764
3,4,Waiting to Exhale (1995),Comedy|Drama,0.038394
4,5,Father of the Bride Part II (1995),Comedy,0.060639


In [16]:
# 训练集预测，求得RMSE
# 将dataframe形式的用户物品相似矩阵转为numpy 数组
rating_matrix_array = movie_rating_matrix.values
user_item_pre = rating_matrix_array.dot(movie_similarity) / np.array([np.abs(movie_similarity).sum(axis = 1)])

pre_flatten = user_item_pre
rating_matrix_flatten = rating_matrix_array
from sklearn.metrics import mean_squared_error
from math import sqrt
error_train = sqrt(mean_squared_error(pre_flatten, rating_matrix_flatten))
print(error_train)


0.050299778955169176


In [17]:
# 测试集预测，求得RMSE
rating_test_matrix_array = rating_matrix_test.values
user_item_test_pre = rating_test_matrix_array.dot(movie_similarity_test) / np.array([np.abs(movie_similarity_test).sum(axis = 1)])

test_pre_flatten = user_item_test_pre
test_rating_matrix_flatten = rating_test_matrix_array
error_test = sqrt(mean_squared_error(test_pre_flatten, test_rating_matrix_flatten))
print(error_test)

0.025515003048719793
